# Combine multi year data files and integrate into 1 dataset

In [1]:
%pip install swifter

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.1.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
"""
CORRECTED DATA INTEGRATION PIPELINE
Training: 2020-2023 | Testing: 2024
"""

import pandas as pd
import numpy as np
import os
from datetime import timedelta
from tqdm import tqdm
import pickle
import warnings
warnings.filterwarnings('ignore')

print("="*70)
print("🚀 VOYAGE DELAY PREDICTION - DATA INTEGRATION")
print("="*70)

# ============================================================================
# CONFIGURATION
# ============================================================================

TRAIN_YEARS = [2020, 2021, 2022, 2023]
TEST_YEARS = [2024]

WEATHER_BUFFER = 5
NEWS_BUFFER = 10

OUTPUT_TRAIN = 'train_2020_2023.csv'
OUTPUT_TEST = 'test_2024.csv'
PREPROCESSING_PARAMS = 'preprocessing_params.pkl'

print(f"\n📁 Configuration:")
print(f"   Training years: {TRAIN_YEARS}")
print(f"   Test years: {TEST_YEARS}")

🚀 VOYAGE DELAY PREDICTION - DATA INTEGRATION

📁 Configuration:
   Training years: [2020, 2021, 2022, 2023]
   Test years: [2024]


In [2]:
# ============================================================================
# LOAD AND COMBINE TRAINING DATA (2020-2023)
# ============================================================================

print("\n" + "="*70)
print("STEP 1: LOADING TRAINING DATA (2020-2023)")
print("="*70)

def load_ais_data(years):
    dfs = []
    for year in years:
        filename = f'ais_data_{year}_cleaned.csv'
        if os.path.exists(filename):
            print(f"📂 Loading {filename}...")
            df = pd.read_csv(filename, parse_dates=['StartTime', 'EndTime'],
                           dtype={'MMSI': 'str', 'VesselName': 'str', 'IMO': 'str'})
            dfs.append(df)
            print(f"   ✅ {len(df):,} voyages")
    return pd.concat(dfs, ignore_index=True) if dfs else pd.DataFrame()

def load_weather_data(years):
    dfs = []
    for year in years:
        filename = f'era5_weather_{year}_sampled.csv'
        if os.path.exists(filename):
            print(f"📂 Loading {filename}...")
            df = pd.read_csv(filename, parse_dates=['time'])
            dfs.append(df)
            print(f"   ✅ {len(df):,} records")
    return pd.concat(dfs, ignore_index=True) if dfs else pd.DataFrame()

def load_gdelt_data(years):
    dfs = []
    for year in years:
        filename = f'gdelt_events_clean_{year}.csv'
        if os.path.exists(filename):
            print(f"📂 Loading {filename}...")
            df = pd.read_csv(filename, parse_dates=['Date'])
            dfs.append(df)
            print(f"   ✅ {len(df):,} events")
    return pd.concat(dfs, ignore_index=True) if dfs else pd.DataFrame()

print("\n🔵 TRAINING DATA:")
train_ais = load_ais_data(TRAIN_YEARS)
train_weather = load_weather_data(TRAIN_YEARS)
train_gdelt = load_gdelt_data(TRAIN_YEARS)

print(f"\n✅ Training data loaded:")
print(f"   AIS: {len(train_ais):,} voyages")
print(f"   Weather: {len(train_weather):,} records")
print(f"   GDELT: {len(train_gdelt):,} events")


STEP 1: LOADING TRAINING DATA (2020-2023)

🔵 TRAINING DATA:
📂 Loading ais_data_2020_cleaned.csv...
   ✅ 296,850 voyages
📂 Loading ais_data_2021_cleaned.csv...
   ✅ 324,672 voyages
📂 Loading ais_data_2022_cleaned.csv...
   ✅ 334,365 voyages
📂 Loading ais_data_2023_cleaned.csv...
   ✅ 352,334 voyages
📂 Loading era5_weather_2020_sampled.csv...
   ✅ 1,354,200 records
📂 Loading era5_weather_2021_sampled.csv...
   ✅ 1,350,500 records
📂 Loading era5_weather_2022_sampled.csv...
   ✅ 1,350,500 records
📂 Loading era5_weather_2023_sampled.csv...
   ✅ 1,350,500 records
📂 Loading gdelt_events_clean_2020.csv...
   ✅ 442,397 events
📂 Loading gdelt_events_clean_2021.csv...
   ✅ 405,830 events
📂 Loading gdelt_events_clean_2022.csv...
   ✅ 323,349 events
📂 Loading gdelt_events_clean_2023.csv...
   ✅ 445,836 events

✅ Training data loaded:
   AIS: 1,308,221 voyages
   Weather: 5,405,700 records
   GDELT: 1,617,412 events


In [3]:
# ============================================================================
# PREPROCESS TRAINING DATA & SAVE PARAMETERS
# ============================================================================

print("\n" + "="*70)
print("STEP 2: PREPROCESSING TRAINING DATA")
print("="*70)

preprocessing_params = {}

# ========== WEATHER PREPROCESSING ==========
print("\n🌦️  Preprocessing weather data...")

# Remove invalids
train_weather = train_weather[
    train_weather['latitude'].between(-90, 90) &
    train_weather['longitude'].between(-180, 180)
]

# Calculate statistics ONLY on training data
weather_means = train_weather.select_dtypes(include=[np.number]).mean()
preprocessing_params['weather_means'] = weather_means

# Fill missing values
train_weather = train_weather.fillna(weather_means)

# Calculate outlier bounds ONLY on training data
weather_cols = ['wind_speed', 't2m', 'sp', 'msl', 'tp']
outlier_bounds = {}
for col in weather_cols:
    Q1 = train_weather[col].quantile(0.01)
    Q3 = train_weather[col].quantile(0.99)
    outlier_bounds[col] = (Q1, Q3)
    train_weather[col] = train_weather[col].clip(Q1, Q3)
preprocessing_params['weather_outlier_bounds'] = outlier_bounds

# Feature engineering
train_weather['temp_celsius'] = train_weather['t2m'] - 273.15
train_weather['pressure_hpa'] = train_weather['sp'] / 100
train_weather['hour'] = train_weather['time'].dt.hour
train_weather['day_of_year'] = train_weather['time'].dt.dayofyear

print(f"   ✅ Weather preprocessing complete: {len(train_weather):,} records")

# ========== GDELT PREPROCESSING ==========
print("\n📰 Preprocessing GDELT data...")

train_gdelt['Actor1CountryCode'] = train_gdelt['Actor1CountryCode'].fillna('UNKNOWN')
train_gdelt = train_gdelt[
    train_gdelt['GoldsteinScale'].between(-10, 10) &
    train_gdelt['ActionGeo_Lat'].between(-90, 90) &
    train_gdelt['ActionGeo_Long'].between(-180, 180)
]

def categorize_event(goldstein):
    if goldstein < -5: return 'Very Negative'
    elif goldstein < 0: return 'Negative'
    elif goldstein == 0: return 'Neutral'
    elif goldstein < 5: return 'Positive'
    else: return 'Very Positive'

train_gdelt['EventCategory'] = train_gdelt['GoldsteinScale'].apply(categorize_event)

print(f"   ✅ GDELT preprocessing complete: {len(train_gdelt):,} events")

# Save preprocessing parameters
print(f"\n💾 Saving preprocessing parameters to: {PREPROCESSING_PARAMS}")
with open(PREPROCESSING_PARAMS, 'wb') as f:
    pickle.dump(preprocessing_params, f)
print("   ✅ Saved!")


STEP 2: PREPROCESSING TRAINING DATA

🌦️  Preprocessing weather data...
   ✅ Weather preprocessing complete: 5,405,700 records

📰 Preprocessing GDELT data...
   ✅ GDELT preprocessing complete: 1,617,412 events

💾 Saving preprocessing parameters to: preprocessing_params.pkl
   ✅ Saved!


In [4]:
# ============================================================================
# FEATURE ENGINEERING FOR TRAINING DATA
# ============================================================================

print("\n" + "="*70)
print("STEP 3: FEATURE ENGINEERING - TRAINING DATA")
print("="*70)

# ========== STEP 3.1: CREATE SPATIAL-TEMPORAL INDICES ==========
print("\n🔧 Creating spatial-temporal indices for faster lookups...")

# Sort and create time bins for weather data
train_weather = train_weather.sort_values('time').reset_index(drop=True)
train_weather['year_month'] = train_weather['time'].dt.to_period('M')

# Create spatial bins for weather (1-degree grid)
train_weather['lat_bin'] = (train_weather['latitude'] // 1).astype(int)
train_weather['lon_bin'] = (train_weather['longitude'] // 1).astype(int)

# Sort and create date bins for GDELT
train_gdelt = train_gdelt.sort_values('Date').reset_index(drop=True)
train_gdelt['year_month'] = train_gdelt['Date'].dt.to_period('M')

# Create spatial bins for GDELT (1-degree grid)
train_gdelt['lat_bin'] = (train_gdelt['ActionGeo_Lat'] // 1).astype(int)
train_gdelt['lon_bin'] = (train_gdelt['ActionGeo_Long'] // 1).astype(int)

print("   ✅ Indices created")

# ========== STEP 3.2: FEATURE EXTRACTION FUNCTION ==========

def extract_voyage_features_optimized(row):
    """Extract weather and news features along ACTUAL route (start → end)"""

    start = row['StartTime']
    end = row['EndTime']
    start_lat, start_lon = row['StartLatitude'], row['StartLongitude']
    end_lat, end_lon = row['EndLatitude'], row['EndLongitude']

    # Create bounding box along ACTUAL route
    min_lat = min(start_lat, end_lat) - WEATHER_BUFFER
    max_lat = max(start_lat, end_lat) + WEATHER_BUFFER
    min_lon = min(start_lon, end_lon) - WEATHER_BUFFER
    max_lon = max(start_lon, end_lon) + WEATHER_BUFFER

    # ========== WEATHER FEATURES ==========
    # Step 1: Filter by time period (coarse filter)
    voyage_period = start.to_period('M')
    period_range = [voyage_period]
    if start.to_period('M') != end.to_period('M'):
        period_range.append(end.to_period('M'))
    
    weather_time_filtered = train_weather[
        train_weather['year_month'].isin(period_range)
    ]
    
    # Step 2: Filter by spatial bins (coarse filter)
    lat_bins = range(int(min_lat) - 1, int(max_lat) + 2)
    lon_bins = range(int(min_lon) - 1, int(max_lon) + 2)
    
    if len(weather_time_filtered) > 0:
        weather_spatial_filtered = weather_time_filtered[
            weather_time_filtered['lat_bin'].isin(lat_bins) &
            weather_time_filtered['lon_bin'].isin(lon_bins)
        ]
        
        # Step 3: Fine-grained filter (only on subset)
        if len(weather_spatial_filtered) > 0:
            weather_mask = (
                (weather_spatial_filtered['time'] >= start) & 
                (weather_spatial_filtered['time'] <= end) &
                (weather_spatial_filtered['latitude'].between(min_lat, max_lat)) &
                (weather_spatial_filtered['longitude'].between(min_lon, max_lon))
            )
            voyage_weather = weather_spatial_filtered[weather_mask]
        else:
            voyage_weather = pd.DataFrame()
    else:
        voyage_weather = pd.DataFrame()
    
    # Extract weather features
    if len(voyage_weather) > 0:
        weather_features = {
            'avg_wind_speed': voyage_weather['wind_speed'].mean(),
            'max_wind_speed': voyage_weather['wind_speed'].max(),
            'min_wind_speed': voyage_weather['wind_speed'].min(),
            'avg_temp_celsius': voyage_weather['temp_celsius'].mean(),
            'max_temp_celsius': voyage_weather['temp_celsius'].max(),
            'min_temp_celsius': voyage_weather['temp_celsius'].min(),
            'avg_pressure_hpa': voyage_weather['pressure_hpa'].mean(),
            'total_precipitation': voyage_weather['tp'].sum(),
            'weather_records': len(voyage_weather)
        }
    else:
        weather_features = {
            'avg_wind_speed': np.nan, 'max_wind_speed': np.nan, 'min_wind_speed': np.nan,
            'avg_temp_celsius': np.nan, 'max_temp_celsius': np.nan, 'min_temp_celsius': np.nan,
            'avg_pressure_hpa': np.nan, 'total_precipitation': np.nan, 'weather_records': 0
        }
    
    # ========== NEWS FEATURES ==========
    min_lat_news = min(start_lat, end_lat) - NEWS_BUFFER
    max_lat_news = max(start_lat, end_lat) + NEWS_BUFFER
    min_lon_news = min(start_lon, end_lon) - NEWS_BUFFER
    max_lon_news = max(start_lon, end_lon) + NEWS_BUFFER
    
    # Step 1: Filter by time period
    gdelt_time_filtered = train_gdelt[
        train_gdelt['year_month'].isin(period_range)
    ]
    
    # Step 2: Filter by spatial bins
    lat_bins_news = range(int(min_lat_news) - 1, int(max_lat_news) + 2)
    lon_bins_news = range(int(min_lon_news) - 1, int(max_lon_news) + 2)
    
    if len(gdelt_time_filtered) > 0:
        gdelt_spatial_filtered = gdelt_time_filtered[
            gdelt_time_filtered['lat_bin'].isin(lat_bins_news) &
            gdelt_time_filtered['lon_bin'].isin(lon_bins_news)
        ]
        
        # Step 3: Fine-grained filter
        if len(gdelt_spatial_filtered) > 0:
            news_mask = (
                (gdelt_spatial_filtered['Date'] >= start) & 
                (gdelt_spatial_filtered['Date'] <= end) &
                (gdelt_spatial_filtered['ActionGeo_Lat'].between(min_lat_news, max_lat_news)) &
                (gdelt_spatial_filtered['ActionGeo_Long'].between(min_lon_news, max_lon_news))
            )
            voyage_news = gdelt_spatial_filtered[news_mask]
        else:
            voyage_news = pd.DataFrame()
    else:
        voyage_news = pd.DataFrame()
    
    # Extract news features
    if len(voyage_news) > 0:
        news_features = {
            'num_events': len(voyage_news),
            'avg_goldstein': voyage_news['GoldsteinScale'].mean(),
            'min_goldstein': voyage_news['GoldsteinScale'].min(),
            'max_goldstein': voyage_news['GoldsteinScale'].max(),
            'avg_tone': voyage_news['AvgTone'].mean(),
            'total_mentions': voyage_news['NumMentions'].sum(),
            'total_sources': voyage_news['NumSources'].sum(),
            'negative_events': (voyage_news['GoldsteinScale'] < 0).sum(),
            'positive_events': (voyage_news['GoldsteinScale'] > 0).sum(),
        }
    else:
        news_features = {
            'num_events': 0, 'avg_goldstein': 0, 'min_goldstein': 0, 'max_goldstein': 0,
            'avg_tone': 0, 'total_mentions': 0, 'total_sources': 0,
            'negative_events': 0, 'positive_events': 0
        }
    
    return {**weather_features, **news_features}

# ========== STEP 3.3: FUNCTION PROCESSING USING SWIFTER ==========

import swifter

# ============= TEST SAMPLE RUN =============
'''
SAMPLE_SIZE = 50000

print(f"\n🧪 TESTING MODE: Processing {SAMPLE_SIZE:,} sample voyages...")

train_ais_sample = train_ais.head(SAMPLE_SIZE).copy()

print(f"   Original dataset: {len(train_ais):,} voyages")
print(f"   Sample size: {len(train_ais_sample):,} voyages ({len(train_ais_sample)/len(train_ais)*100:.1f}%)")

# Process sample
train_features = train_ais_sample.swifter.apply(
    lambda row: extract_voyage_features_optimized(row),
    axis=1, result_type='expand'
)

# Combine with original AIS data
train_df = pd.concat([train_ais_sample.reset_index(drop=True), train_features.reset_index(drop=True)], axis=1)
'''
# ============= PRODUCTION RUN =============

print(f"\n🔄 Processing {len(train_ais):,} training voyages...")

# Use swifter
train_features = train_ais.swifter.apply(
    lambda row: extract_voyage_features_optimized(row),
    axis=1, result_type='expand'
)

# Combine with original AIS data
train_df = pd.concat([train_ais.reset_index(drop=True), train_features.reset_index(drop=True)], axis=1)

print(f"\n✅ Feature extraction complete!")
print(f"   Total features extracted: {train_features.shape[1]}")

# ========== STEP 3.4: CREATE TARGET VARIABLES AND ADDITIONAL FEATURES ==========

# Route-based delay definition
print("\n🧭 Creating route-based features and target variables...")

# Create start and end region bins (10-degree grid for broader regions)
train_df['start_lat_region'] = (train_df['StartLatitude'] // 10).astype(int)
train_df['start_lon_region'] = (train_df['StartLongitude'] // 10).astype(int)
train_df['end_lat_region'] = (train_df['EndLatitude'] // 10).astype(int)
train_df['end_lon_region'] = (train_df['EndLongitude'] // 10).astype(int)

# Calculate route-specific baseline durations (median per route)
route_baseline_durations = train_df.groupby([
    'start_lat_region', 'start_lon_region',
    'end_lat_region', 'end_lon_region'
])['DurationHours'].quantile(0.5).to_dict()  # 50th percentile

# Save to preprocessing params for test set
preprocessing_params['route_baseline_durations'] = route_baseline_durations
print(f"   ✅ Calculated baselines for {len(route_baseline_durations)} unique routes")

# Calculate expected duration for each voyage based on its route
print("   🚀 Mapping route baselines...")

# Convert dictionary to DataFrame for fast merging
route_baseline_df = pd.DataFrame([
    {
        'start_lat_region': k[0],
        'start_lon_region': k[1], 
        'end_lat_region': k[2],
        'end_lon_region': k[3],
        'route_baseline': v
    }
    for k, v in route_baseline_durations.items()
])

# Vectorized merge to assign route baselines
train_df = train_df.merge(
    route_baseline_df,
    on=['start_lat_region', 'start_lon_region', 'end_lat_region', 'end_lon_region'],
    how='left'
)

# If route not seen before, use overall median duration
global_median = train_df['DurationHours'].median()
train_df['ExpectedDuration'] = train_df['route_baseline'].fillna(global_median)
train_df = train_df.drop('route_baseline', axis=1)

# Validation checks
print("   🔍 Validation:")
missing_routes = train_df['ExpectedDuration'].isna().sum()
print(f"      Matched routes: {len(train_df) - missing_routes:,} ({(1-missing_routes/len(train_df))*100:.1f}%)")
print(f"      Using fallback: {missing_routes:,} ({missing_routes/len(train_df)*100:.1f}%)")

# Sanity check: Expected duration should be reasonable
invalid_durations = (train_df['ExpectedDuration'] < 0) | (train_df['ExpectedDuration'] > 10000)
if invalid_durations.any():
    print(f"      ⚠️  WARNING: {invalid_durations.sum()} invalid expected durations!")
else:
    print(f"      ✅ All expected durations valid")
    print(f"      📊 Duration range: {train_df['ExpectedDuration'].min():.1f}h - {train_df['ExpectedDuration'].max():.1f}h")

# Target variables
train_df['DurationDifference'] = train_df['DurationHours'] - train_df['ExpectedDuration']
train_df['DelayPercentage'] = (train_df['DurationDifference'] / train_df['ExpectedDuration']) * 100
train_df['IsDelayed'] = (train_df['DelayPercentage'] > 30).astype(int)  # 30% threshold
train_df['DelayHours'] = train_df['DurationDifference']

print(f"   ✅ Target variables created with route-based baselines")
print(f"   📊 New class distribution:")
print(f"      Not Delayed: {(train_df['IsDelayed']==0).sum():,} ({(train_df['IsDelayed']==0).mean()*100:.1f}%)")
print(f"      Delayed: {(train_df['IsDelayed']==1).sum():,} ({(train_df['IsDelayed']==1).mean()*100:.1f}%)")

# Temporal features
train_df['StartMonth'] = train_df['StartTime'].dt.month
train_df['StartDayOfWeek'] = train_df['StartTime'].dt.dayofweek
train_df['StartHour'] = train_df['StartTime'].dt.hour
train_df['StartQuarter'] = train_df['StartTime'].dt.quarter
train_df['StartYear'] = train_df['StartTime'].dt.year

# Weather severity flags
train_df['HighWindFlag'] = (train_df['max_wind_speed'] > 15).astype(int)
train_df['HeavyPrecipitationFlag'] = (train_df['total_precipitation'] > 0.01).astype(int)

# News sentiment indicators
train_df['HasNegativeNews'] = (train_df['negative_events'] > 0).astype(int)
train_df['NegativeNewsRatio'] = train_df['negative_events'] / (train_df['num_events'] + 1)

# Fill missing values
weather_news_cols = [col for col in train_df.columns if col.startswith(('avg_', 'max_', 'min_', 'total_'))]
train_df[weather_news_cols] = train_df[weather_news_cols].fillna(0)

print("   ✅ All features created")

# ========== STEP 3.5: SAVE TRAINING DATA ==========

print(f"\n💾 Saving training data to: {OUTPUT_TRAIN}")
train_df.to_csv(OUTPUT_TRAIN, index=False)

file_size = os.path.getsize(OUTPUT_TRAIN) / (1024**2)
print(f"   ✅ Saved successfully!")
print(f"   📊 Rows: {len(train_df):,}")
print(f"   📋 Columns: {train_df.shape[1]}")
print(f"   📁 File size: {file_size:.1f} MB")
print(f"   🎯 Delayed voyages: {train_df['IsDelayed'].sum():,} ({train_df['IsDelayed'].mean()*100:.1f}%)")


STEP 3: FEATURE ENGINEERING - TRAINING DATA

🔧 Creating spatial-temporal indices for faster lookups...
   ✅ Indices created

🔄 Processing 1,308,221 training voyages...


Pandas Apply: 100%|██████████| 1308221/1308221 [16:22:24<00:00, 22.19it/s]   



✅ Feature extraction complete!
   Total features extracted: 18

🧭 Creating route-based features and target variables...
   ✅ Calculated baselines for 1494 unique routes
   🚀 Mapping route baselines...
   🔍 Validation:
      Matched routes: 1,308,221 (100.0%)
      Using fallback: 0 (0.0%)
      ✅ All expected durations valid
      📊 Duration range: 0.6h - 744.0h
   ✅ Target variables created with route-based baselines
   📊 New class distribution:
      Not Delayed: 975,604 (74.6%)
      Delayed: 332,617 (25.4%)
   ✅ All features created

💾 Saving training data to: train_2020_2023.csv
   ✅ Saved successfully!
   📊 Rows: 1,308,221
   📋 Columns: 49
   📁 File size: 441.3 MB
   🎯 Delayed voyages: 332,617 (25.4%)


In [5]:
# ============================================================================
# LOAD AND PREPROCESS TEST DATA (2024)
# ============================================================================

print("\n" + "="*70)
print("STEP 4: PROCESSING TEST DATA (2024)")
print("="*70)

print("\n🔴 TEST DATA:")
test_ais = load_ais_data(TEST_YEARS)
test_weather = load_weather_data(TEST_YEARS)
test_gdelt = load_gdelt_data(TEST_YEARS)

print(f"\n✅ Test data loaded:")
print(f"   AIS: {len(test_ais):,} voyages")
print(f"   Weather: {len(test_weather):,} records")
print(f"   GDELT: {len(test_gdelt):,} events")

# ========== APPLY TRAINING PREPROCESSING TO TEST DATA ==========
print("\n📋 Loading preprocessing parameters...")
with open(PREPROCESSING_PARAMS, 'rb') as f:
    params = pickle.load(f)

print("\n🌦️  Applying weather preprocessing...")
test_weather = test_weather[
    test_weather['latitude'].between(-90, 90) &
    test_weather['longitude'].between(-180, 180)
]
test_weather = test_weather.fillna(params['weather_means'])

for col, (Q1, Q3) in params['weather_outlier_bounds'].items():
    test_weather[col] = test_weather[col].clip(Q1, Q3)

test_weather['temp_celsius'] = test_weather['t2m'] - 273.15
test_weather['pressure_hpa'] = test_weather['sp'] / 100
test_weather['hour'] = test_weather['time'].dt.hour
test_weather['day_of_year'] = test_weather['time'].dt.dayofyear

print(f"   ✅ Weather preprocessing complete: {len(test_weather):,} records")

print("\n📰 Applying GDELT preprocessing...")
test_gdelt['Actor1CountryCode'] = test_gdelt['Actor1CountryCode'].fillna('UNKNOWN')
test_gdelt = test_gdelt[
    test_gdelt['GoldsteinScale'].between(-10, 10) &
    test_gdelt['ActionGeo_Lat'].between(-90, 90) &
    test_gdelt['ActionGeo_Long'].between(-180, 180)
]
test_gdelt['EventCategory'] = test_gdelt['GoldsteinScale'].apply(categorize_event)
print(f"   ✅ GDELT preprocessing complete: {len(test_gdelt):,} events")

# ========== CREATE INDICES FOR TEST DATA ==========
print("\n🔧 Creating spatial-temporal indices for test data...")

test_weather = test_weather.sort_values('time').reset_index(drop=True)
test_weather['year_month'] = test_weather['time'].dt.to_period('M')
test_weather['lat_bin'] = (test_weather['latitude'] // 1).astype(int)
test_weather['lon_bin'] = (test_weather['longitude'] // 1).astype(int)

test_gdelt = test_gdelt.sort_values('Date').reset_index(drop=True)
test_gdelt['year_month'] = test_gdelt['Date'].dt.to_period('M')
test_gdelt['lat_bin'] = (test_gdelt['ActionGeo_Lat'] // 1).astype(int)
test_gdelt['lon_bin'] = (test_gdelt['ActionGeo_Long'] // 1).astype(int)

print("   ✅ Test indices created")

# ========== FEATURE EXTRACTION FUNCTION FOR TEST DATA ==========

def extract_test_features_optimized(row):
    """Extract weather and news features along ACTUAL route for test data"""
    
    start = row['StartTime']
    end = row['EndTime']
    start_lat, start_lon = row['StartLatitude'], row['StartLongitude']
    end_lat, end_lon = row['EndLatitude'], row['EndLongitude']
    
    # Create bounding box along ACTUAL route
    min_lat = min(start_lat, end_lat) - WEATHER_BUFFER
    max_lat = max(start_lat, end_lat) + WEATHER_BUFFER
    min_lon = min(start_lon, end_lon) - WEATHER_BUFFER
    max_lon = max(start_lon, end_lon) + WEATHER_BUFFER
    
    # Weather features
    voyage_period = start.to_period('M')
    period_range = [voyage_period]
    if start.to_period('M') != end.to_period('M'):
        period_range.append(end.to_period('M'))
    
    weather_time_filtered = test_weather[test_weather['year_month'].isin(period_range)]
    
    lat_bins = range(int(min_lat) - 1, int(max_lat) + 2)
    lon_bins = range(int(min_lon) - 1, int(max_lon) + 2)
    
    if len(weather_time_filtered) > 0:
        weather_spatial_filtered = weather_time_filtered[
            weather_time_filtered['lat_bin'].isin(lat_bins) &
            weather_time_filtered['lon_bin'].isin(lon_bins)
        ]
        
        if len(weather_spatial_filtered) > 0:
            weather_mask = (
                (weather_spatial_filtered['time'] >= start) & 
                (weather_spatial_filtered['time'] <= end) &
                (weather_spatial_filtered['latitude'].between(min_lat, max_lat)) &
                (weather_spatial_filtered['longitude'].between(min_lon, max_lon))
            )
            voyage_weather = weather_spatial_filtered[weather_mask]
        else:
            voyage_weather = pd.DataFrame()
    else:
        voyage_weather = pd.DataFrame()
    
    if len(voyage_weather) > 0:
        weather_features = {
            'avg_wind_speed': voyage_weather['wind_speed'].mean(),
            'max_wind_speed': voyage_weather['wind_speed'].max(),
            'min_wind_speed': voyage_weather['wind_speed'].min(),
            'avg_temp_celsius': voyage_weather['temp_celsius'].mean(),
            'max_temp_celsius': voyage_weather['temp_celsius'].max(),
            'min_temp_celsius': voyage_weather['temp_celsius'].min(),
            'avg_pressure_hpa': voyage_weather['pressure_hpa'].mean(),
            'total_precipitation': voyage_weather['tp'].sum(),
            'weather_records': len(voyage_weather)
        }
    else:
        weather_features = {
            'avg_wind_speed': np.nan, 'max_wind_speed': np.nan, 'min_wind_speed': np.nan,
            'avg_temp_celsius': np.nan, 'max_temp_celsius': np.nan, 'min_temp_celsius': np.nan,
            'avg_pressure_hpa': np.nan, 'total_precipitation': np.nan, 'weather_records': 0
        }
    
    # News features
    min_lat_news = min(start_lat, end_lat) - NEWS_BUFFER
    max_lat_news = max(start_lat, end_lat) + NEWS_BUFFER
    min_lon_news = min(start_lon, end_lon) - NEWS_BUFFER
    max_lon_news = max(start_lon, end_lon) + NEWS_BUFFER
    
    gdelt_time_filtered = test_gdelt[test_gdelt['year_month'].isin(period_range)]
    
    lat_bins_news = range(int(min_lat_news) - 1, int(max_lat_news) + 2)
    lon_bins_news = range(int(min_lon_news) - 1, int(max_lon_news) + 2)
    
    if len(gdelt_time_filtered) > 0:
        gdelt_spatial_filtered = gdelt_time_filtered[
            gdelt_time_filtered['lat_bin'].isin(lat_bins_news) &
            gdelt_time_filtered['lon_bin'].isin(lon_bins_news)
        ]
        
        if len(gdelt_spatial_filtered) > 0:
            news_mask = (
                (gdelt_spatial_filtered['Date'] >= start) & 
                (gdelt_spatial_filtered['Date'] <= end) &
                (gdelt_spatial_filtered['ActionGeo_Lat'].between(min_lat_news, max_lat_news)) &
                (gdelt_spatial_filtered['ActionGeo_Long'].between(min_lon_news, max_lon_news))
            )
            voyage_news = gdelt_spatial_filtered[news_mask]
        else:
            voyage_news = pd.DataFrame()
    else:
        voyage_news = pd.DataFrame()
    
    if len(voyage_news) > 0:
        news_features = {
            'num_events': len(voyage_news),
            'avg_goldstein': voyage_news['GoldsteinScale'].mean(),
            'min_goldstein': voyage_news['GoldsteinScale'].min(),
            'max_goldstein': voyage_news['GoldsteinScale'].max(),
            'avg_tone': voyage_news['AvgTone'].mean(),
            'total_mentions': voyage_news['NumMentions'].sum(),
            'total_sources': voyage_news['NumSources'].sum(),
            'negative_events': (voyage_news['GoldsteinScale'] < 0).sum(),
            'positive_events': (voyage_news['GoldsteinScale'] > 0).sum(),
        }
    else:
        news_features = {
            'num_events': 0, 'avg_goldstein': 0, 'min_goldstein': 0, 'max_goldstein': 0,
            'avg_tone': 0, 'total_mentions': 0, 'total_sources': 0,
            'negative_events': 0, 'positive_events': 0
        }
    
    return {**weather_features, **news_features}

# ========== FEATURE ENGINEERING FOR TEST DATA ==========

import swifter

# ============= TEST SAMPLE RUN =============
'''
TEST_SAMPLE_SIZE = 10000
test_ais_sample = test_ais.head(TEST_SAMPLE_SIZE).copy()

print(f"\n🔄 Processing {len(test_ais_sample):,} test voyages (sample)...")
test_features = test_ais_sample.swifter.apply(
    lambda row: extract_test_features_optimized(row),
    axis=1, result_type='expand'
)

test_df = pd.concat([test_ais_sample.reset_index(drop=True), test_features.reset_index(drop=True)], axis=1)
'''
# ============ PRODUCTION RUN =============

print(f"\n🔄 Processing {len(test_ais):,} test voyages...")
test_features = test_ais.swifter.apply(
    lambda row: extract_test_features_optimized(row),
    axis=1, result_type='expand'
)

test_df = pd.concat([test_ais.reset_index(drop=True), test_features.reset_index(drop=True)], axis=1)

# Apply route-based approach (same as training)
print("🧭 Creating route-based features for test set...")

# Create region bins
test_df['start_lat_region'] = (test_df['StartLatitude'] // 10).astype(int)
test_df['start_lon_region'] = (test_df['StartLongitude'] // 10).astype(int)
test_df['end_lat_region'] = (test_df['EndLatitude'] // 10).astype(int)
test_df['end_lon_region'] = (test_df['EndLongitude'] // 10).astype(int)

# Calculate expected duration based on route
print("   🚀 Mapping route baselines for test set...")

# Use the same route_baseline_df from training
test_df = test_df.merge(
    route_baseline_df,
    on=['start_lat_region', 'start_lon_region', 'end_lat_region', 'end_lon_region'],
    how='left'
)

# For unseen routes, use median of all test voyages with similar start region
unseen_routes_mask = test_df['route_baseline'].isna()
unseen_count = unseen_routes_mask.sum()

if unseen_count > 0:
    print(f"   ⚠️  Found {unseen_count:,} voyages with unseen routes")
    
    # Calculate start region medians for fallback
    start_region_medians = test_df.groupby(
        ['start_lat_region', 'start_lon_region']
    )['DurationHours'].transform('median')
    
    # Apply fallback: start region median, then global median
    test_df.loc[unseen_routes_mask, 'route_baseline'] = start_region_medians[unseen_routes_mask]
    
    # For routes with no similar start region, use global median
    still_missing = test_df['route_baseline'].isna()
    if still_missing.any():
        test_df.loc[still_missing, 'route_baseline'] = test_df['DurationHours'].median()

# Rename column
test_df['ExpectedDuration'] = test_df['route_baseline']
test_df = test_df.drop('route_baseline', axis=1)

# Validation checks
print("   🔍 Validation:")
matched_routes = len(test_df) - unseen_count
print(f"      Matched training routes: {matched_routes:,} ({matched_routes/len(test_df)*100:.1f}%)")
print(f"      Unseen routes (fallback): {unseen_count:,} ({unseen_count/len(test_df)*100:.1f}%)")

# Sanity check
invalid_durations = (test_df['ExpectedDuration'] < 0) | (test_df['ExpectedDuration'] > 10000)
if invalid_durations.any():
    print(f"      ⚠️  WARNING: {invalid_durations.sum()} invalid expected durations!")
else:
    print(f"      ✅ All expected durations valid")
    print(f"      📊 Duration range: {test_df['ExpectedDuration'].min():.1f}h - {test_df['ExpectedDuration'].max():.1f}h")

# Target variables
test_df['DurationDifference'] = test_df['DurationHours'] - test_df['ExpectedDuration']
test_df['DelayPercentage'] = (test_df['DurationDifference'] / test_df['ExpectedDuration']) * 100
test_df['IsDelayed'] = (test_df['DelayPercentage'] > 30).astype(int)  # Same 30% threshold
test_df['DelayHours'] = test_df['DurationDifference']

print(f"   ✅ Test set targets created")
print(f"   📊 Class distribution:")
print(f"      Not Delayed: {(test_df['IsDelayed']==0).sum():,} ({(test_df['IsDelayed']==0).mean()*100:.1f}%)")
print(f"      Delayed: {(test_df['IsDelayed']==1).sum():,} ({(test_df['IsDelayed']==1).mean()*100:.1f}%)")

# Additional features (same as training)
test_df['StartMonth'] = test_df['StartTime'].dt.month
test_df['StartDayOfWeek'] = test_df['StartTime'].dt.dayofweek
test_df['StartHour'] = test_df['StartTime'].dt.hour
test_df['StartQuarter'] = test_df['StartTime'].dt.quarter
test_df['StartYear'] = test_df['StartTime'].dt.year
test_df['HighWindFlag'] = (test_df['max_wind_speed'] > 15).astype(int)
test_df['HeavyPrecipitationFlag'] = (test_df['total_precipitation'] > 0.01).astype(int)
test_df['HasNegativeNews'] = (test_df['negative_events'] > 0).astype(int)
test_df['NegativeNewsRatio'] = test_df['negative_events'] / (test_df['num_events'] + 1)

# Fill missing values
test_df[weather_news_cols] = test_df[weather_news_cols].fillna(0)

print(f"\n💾 Saving test data to: {OUTPUT_TEST}")
test_df.to_csv(OUTPUT_TEST, index=False)
print(f"   ✅ Saved {len(test_df):,} rows, {test_df.shape[1]} columns")


STEP 4: PROCESSING TEST DATA (2024)

🔴 TEST DATA:
📂 Loading ais_data_2024_cleaned.csv...
   ✅ 367,077 voyages
📂 Loading era5_weather_2024_sampled.csv...
   ✅ 1,354,200 records
📂 Loading gdelt_events_clean_2024.csv...
   ✅ 466,495 events

✅ Test data loaded:
   AIS: 367,077 voyages
   Weather: 1,354,200 records
   GDELT: 466,495 events

📋 Loading preprocessing parameters...

🌦️  Applying weather preprocessing...
   ✅ Weather preprocessing complete: 1,354,200 records

📰 Applying GDELT preprocessing...
   ✅ GDELT preprocessing complete: 466,495 events

🔧 Creating spatial-temporal indices for test data...
   ✅ Test indices created

🔄 Processing 367,077 test voyages...


Pandas Apply: 100%|██████████| 367077/367077 [2:18:39<00:00, 44.12it/s]  


🧭 Creating route-based features for test set...
   🚀 Mapping route baselines for test set...
   ⚠️  Found 52 voyages with unseen routes
   🔍 Validation:
      Matched training routes: 367,025 (100.0%)
      Unseen routes (fallback): 52 (0.0%)
      ✅ All expected durations valid
      📊 Duration range: 0.6h - 743.9h
   ✅ Test set targets created
   📊 Class distribution:
      Not Delayed: 275,022 (74.9%)
      Delayed: 92,055 (25.1%)

💾 Saving test data to: test_2024.csv
   ✅ Saved 367,077 rows, 49 columns


In [6]:
# ============================================================================
# FINAL SUMMARY
# ============================================================================

print("\n" + "="*70)
print("🎉 PIPELINE COMPLETED!")
print("="*70)

print(f"\n📊 TRAINING SET (2020-2023):")
print(f"   Voyages: {len(train_df):,}")
print(f"   Delayed: {train_df['IsDelayed'].sum():,} ({train_df['IsDelayed'].mean()*100:.1f}%)")

print(f"\n📊 TEST SET (2024):")
print(f"   Voyages: {len(test_df):,}")
print(f"   Delayed: {test_df['IsDelayed'].sum():,} ({test_df['IsDelayed'].mean()*100:.1f}%)")

print(f"\n✅ Next steps:")
print(f"   1. Load: train = pd.read_csv('{OUTPUT_TRAIN}')")
print(f"   2. Load: test = pd.read_csv('{OUTPUT_TEST}')")
print(f"   3. Train models on train set")
print(f"   4. Evaluate on test set (2024)")

print("\n" + "="*70)


🎉 PIPELINE COMPLETED!

📊 TRAINING SET (2020-2023):
   Voyages: 1,308,221
   Delayed: 332,617 (25.4%)

📊 TEST SET (2024):
   Voyages: 367,077
   Delayed: 92,055 (25.1%)

✅ Next steps:
   1. Load: train = pd.read_csv('train_2020_2023.csv')
   2. Load: test = pd.read_csv('test_2024.csv')
   3. Train models on train set
   4. Evaluate on test set (2024)

